# Trapezoid Method, Simpson's Rule and Romberg Integration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Define a function for taking an integral

In [ ]:
def func(x):
    return np.exp(-2.0*x)*np.cos(10.0*x)

## Define it's integral so we know the right answer

In [ ]:
def func_integral(x):
    return 1/52*np.exp(-2*x)*(5*np.sin(10*x)-np.cos(10*x))

## Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h)+f(x))

## Define a wrapper function for trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    # f == funtion to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    
    #define x value to perform trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    #define in value of the integral
    Fint = 0.0
    
    #perform in integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
    #return the answer
    return Fint

## Define the core of the Simpson's Method

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h) )/3

## Define a wrapper function to perform Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
      # f == funtion to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of intervals
    #note the number of chunks will be N-1
    #so if N is odd, the we don't need to adjust the 
    #last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using simpson's method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        
    #apply simpson's rule over the last interval
    #if N is even
    if ((N%2) == 0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint
        

## Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    #we need the difference b-a
    h = b - a
    
    #and the increment between new function evaluations
    dh = h/2.**(i)
    
    #need the cofactor
    K = h/2.**(i+1)
    
    #and to compute the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

## Define a wrapper function to perform Romberg integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array to integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) +f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        
        #print step
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i+=1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
                
    #return the answer
    return (I[i],i)

## Check the integrals

In [ ]:
Answer = func_integral(np.pi)-func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,np.pi,10))
print("Simpson's Method")
print(simpsons_method(func,0,np.pi,10))
print("Romberg")
tolerance = 1.0e-6

#This romberg integration takes 5 minutes to complete 
#each step taking roughly twice as long as the last

#I think the issue is in this step of the romberg_core:
    #for j in range(2**i):
        #M += f(a + 0.5*dh + j*dh)
        
#for i == 26, the program needs to call the original function
#2^26 = 67,108,864 times

#in total, the program needs to call the original function about 
#2^27 ~ 130 million times 

RI = romberg_integration(func,0,np.pi,tolerance)
print(RI[0], (RI[0]-Answer)/Answer, tolerance)
print("Romberg takes this many steps to complete:")
print(RI[1])

## Discrete bisection search

In [ ]:
def check_initial_values(f, x_min, x_max):
    
    #check out intital guesses
    y_min = f(x_min)
    y_max = f(x_max)
    
    #check that xim and xmax contain a zero crossing
    if(y_min*y_max >= 0.0):
        print("No zero crossing found in the range = ",x_min,x)
        s = "f(%f) = %f(%f) = %f" % (x_mins,y_min,x_max,y_max)
        print(s)
        return 0 
    
    #if we reach this point, the bracket is valid
    #and we will return 3
    return 3

In [ ]:
def bisection_root_finding(f, x_min_start, x_max_start):

    #this function uses bisection to find a root

    x_min = x_min_start  #minimum x in bracket
    x_max = x_max_start  #maximum x in bracket
    x_mid = 0.0          #mid point
    
    y_min = f(x_min)    #function value at x_min
    y_max = f(x_max)     #function value at x_max
    y_mid = 0.0          #function value at mid point

    imax = 10000    #set a max number of iterations
    i = 0           #iteration counter

    #check the initial values
    flag = check_initial_values(f, x_min, x_max)

    if(flag == 0):
        print("Error in bisection_root_finging().")
        raise ValueError('Inital vlues invalid',x_min,x_max)

    #if we reach here, then we need to conduct the search

    #set a new flag
    flag = 1

    #enter a while loop
    while(flag):
        x_mid = (x_min+x_max)//2   #floored mid point
        y_mid = f(x_mid)           #value of y at x_mid
    
        #check if the search has converged on a single value
        if(np.fabs(x_min-x_max)<=1):
            flag = 0 #we are done!
        
        else: 
            #x_mid is not a root
            
            #if the product of the function at the midpoint 
            #and at one of the endpoints is greater than
            #zero, replace this end point
            if(f(x_min)*f(x_mid)>0):
                #replace x_max with x_mid
                x_min = x_mid
            else:
                #replace x_max with x_mid
                x_max = x_mid
        
        #print out the iteration
        print(x_min,f(x_min),x_max,f(x_max))
        
        #count the iteration
        i += 1 
        
        #if we have exceeded the number of iterations
        #then exit
        if(i>=imax):
            print("Exceeded max number of iterations = ",i)
            s = "Min bracket f(%f) = %f" %(x_min,f(x_min))
            print(s)
            s = "Max bracket f(%f) = %f" %(x_max,f(x_max))
            print(s)
            s = "Mid bracket f(%f) = %f" %(x_mid,f(x_mid))
            print(s)
            raise StopIteration('Stopping iterations after ',i)
            
    #we are done
    return x_max


## Function For Zeros

In [ ]:
#define a function that crosses x axis 
#when the integration approx method 
#reaches the specified tolerance

def func_delta(f,a,b,N,value,tol):
    p = f(func,a,b,N)
    return np.fabs((p-value)/value)-tol

## Check Trapezoid method to Tolerance

In [ ]:
tolerance = 1.0e-6
Answer = func_integral(np.pi)-func_integral(0)


#define a function of 1 variable that crosses x axis 
#when the trapezoid method reaches the specified tolerance
def trapezoid_delta(x):
    return func_delta(trapezoid_method,0,np.pi,x,Answer,tolerance)

TI=bisection_root_finding(trapezoid_delta, 2, 10000)
print("The trapezoid method needs this many intervals to reach the specified tolerance:")
print(TI)
print("The value of the trapezoid method using that many intervals is:")
print(trapezoid_method(func,0,np.pi,TI))
print("The true answer is:")
print(Answer)
print("The fractional error is:")
print(trapezoid_delta(TI)+tolerance)
print("The fractional error using one less interval is:")
print(trapezoid_delta(TI-1)+tolerance)


## Check Simpson's Method to tolerance

In [ ]:
tolerance = 1.0e-6
Answer = func_integral(np.pi)-func_integral(0)


#define a function of 1 variable that crosses x axis 
#when Simpson's method reaches the specified tolerance
def simpsons_delta(x):
    return func_delta(simpsons_method,0,np.pi,x,Answer,tolerance)

SI=bisection_root_finding(simpsons_delta, 2, 10000)
print("Simpson's method needs this many intervals to reach the specified tolerance:")
print(SI)
print("The value of Simpson's method using that many intervals is:")
print(simpsons_method(func,0,np.pi,SI))
print("The true answer is:")
print(Answer)
print("The fractional error is:")
print(simpsons_delta(SI)+tolerance)
print("The fractional error using one less interval is:")
print(simpsons_delta(SI-1)+tolerance)